In [ ]:
import json
import pandas as pd
from datasets import Dataset

def load_data(file_path):
    """
    JSON 데이터를 불러와 Pandas DataFrame으로 변환
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        json_data = json.load(f)
        
    data = []
    for item in json_data['data']:
        if 'text' in item and 'keyword' in item:
            data.append({
                'text': item['text'],
                'keyword': item['keyword']
            })
    
    df = pd.DataFrame(data)
    print(df.info())
    # None 값을 빈 문자열로 대체
    df = df.fillna('')
    return df


# JSON 파일 경로
file_path_time = "/home/yjtech2/Desktop/yurim/LLM/Data/smell_keyword/time_data.json"
time_df = load_data(file_path_time)

In [ ]:
from sklearn.model_selection import train_test_split

# 데이터를 70:30 비율로 나누기
def split_data(df, train_ratio = 0.8):
    """
    DataFrame을 train과 val로 나눔
    """
    train_df, val_df = train_test_split(df, train_size = train_ratio, random_state = 42, shuffle = True)
    return train_df, val_df

# train, val로 나누기
train_time_df, val_time_df = split_data(time_df)

train_time_df.to_csv('/home/yjtech2/Desktop/yurim/LLM/Data/smell_keyword/train_time_df.csv', index = False)
val_time_df.to_csv('/home/yjtech2/Desktop/yurim/LLM/Data/smell_keyword/val_time_df.csv', index = False)

# 나눈 데이터 확인
print('Time')
print("Train Data:")
print(train_time_df.head())
print("\nValidation Data:")
print(val_time_df.head())

# 데이터 크기 확인
print(f"Train size: {len(train_time_df)}, Validation size: {len(val_time_df)}")


In [ ]:
def safe_convert_to_list(value):
    try:
        # JSON 형식으로 변환 가능한지 확인
        return json.loads(value) if isinstance(value, str) else value
    except json.JSONDecodeError:
        # 변환이 불가능한 경우 그대로 반환
        return value

In [ ]:
import pandas as pd
from datasets import Dataset

train_time_df = pd.read_csv('/home/yjtech2/Desktop/yurim/LLM/Data/smell_keyword/train_time_df.csv')
val_time_df = pd.read_csv('/home/yjtech2/Desktop/yurim/LLM/Data/smell_keyword/val_time_df.csv')


# 데이터셋 전체 변환
if train_time_df['keyword'].dtype == 'object':  # keyword가 문자열인지 확인
    train_time_df['keyword'] = train_time_df['keyword'].apply(safe_convert_to_list)  # 또는 json.loads

# val_df의 'keyword' 컬럼 문자열 -> 리스트 변환
if val_time_df['keyword'].dtype == 'object':  # keyword가 문자열인지 확인
    val_time_df['keyword'] = val_time_df['keyword'].apply(safe_convert_to_list)  # 또는 json.loads

train_time_data_dict = train_time_df.to_dict(orient='list')
time_train_dataset = Dataset.from_dict(train_time_data_dict)
time_train_dataset
print(len(time_train_dataset))

val_time_data_dict = val_time_df.to_dict(orient='list')
time_val_dataset = Dataset.from_dict(val_time_data_dict)
print(len(time_val_dataset))

In [ ]:
import os
import torch
from tqdm import tqdm
from typing import Dict
import time
from datetime import datetime
import numpy as np
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    AdamW,
    DataCollatorForSeq2Seq,
)

from transformers import get_linear_schedule_with_warmup
from torch.cuda.amp import GradScaler, autocast
from mecab import MeCab

class CustomKeyBERTTrainer:
    def __init__(self, model_name: str, **kwargs):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Using device: {self.device}")
        if self.device == "cuda":
            print(f"GPU Model: {torch.cuda.get_device_name(0)}")
            print(f"Available GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
        
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(self.device)
        self.optimizer = AdamW(self.model.parameters(), lr=kwargs.get("learning_rate", 1e-4))
        self.max_length = kwargs.get("max_length", 128)
        self.training_args = kwargs
        self.save_dir = kwargs.get("save_dir", "./time_models")
        self.mecab = MeCab()  # MeCab 초기화
        self.best_model_path = os.path.join(self.save_dir, "pytorch_model.bin")
        self.tokenizer_path = self.save_dir
        
        self.history = {
            'train_loss': [],
            'val_loss': [],
            'epoch_times': [],
            'best_epoch': 0
        }

        os.makedirs(self.save_dir, exist_ok=True)
        
    def _mecab_tokenize(self, text: str) -> str:
        """
        MeCab을 사용하여 입력 텍스트에서 명사, 형용사, 부사를 추출
        """
        pos_tags = self.mecab.pos(text)  # 품사 태그 추출
        # 원하는 품사 필터링: 명사(NNG, NNP), 형용사(VA), 부사(MAG, MAJ)
        keyword = [
            word for word, pos in pos_tags if pos in ("NNG", "NNP", "VA", "MAG", "MAJ")
        ]
        return " ".join(keyword)  # 추출한 단어를 공백으로 연결하여 반환
    
    def preprocess_data(self, examples: Dict) -> Dict:
        # MeCab 형태소 분석과 품사 필터링 적용
        examples["text"] = [self._mecab_tokenize(text) for text in examples["text"]]
        
        # 기존 코드 유지
        inputs = [f"키워드 추출: {text}" for text in examples["text"]]
        model_inputs = self.tokenizer(
            inputs,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors=None  # 텐서 변환을 DataCollator에 맡김
        )

        # 레이블(키워드) 처리
        labels = [", ".join(keyword) if keyword else "" for keyword in examples["keyword"]]
        
        with self.tokenizer.as_target_tokenizer():
            tokenized_labels = self.tokenizer(
                labels,
                max_length=self.max_length,
                padding="max_length",
                truncation=True,
                return_tensors=None  # 텐서 변환을 DataCollator에 맡김
            )

        # -100으로 패딩 토큰을 마스킹
        labels = tokenized_labels["input_ids"]
        for i in range(len(labels)):
            for j in range(len(labels[i])):
                if labels[i][j] == self.tokenizer.pad_token_id:
                    labels[i][j] = -100

        model_inputs["labels"] = labels
        return model_inputs
        
    def save_model_and_tokenizer(self, epoch=None, is_best=False):
        """
        최고 성능 모델만 저장하고 이전 모델을 삭제
        """
        if is_best:
            # 이전 최고 모델 디렉토리 삭제
            if os.path.exists(self.best_model_path):
                print(f"Deleting previous best model at {self.best_model_path}")
                os.system(f"rm -rf {self.best_model_path}")
            
            # 새로운 최고 모델 저장
            save_path = os.path.join(self.save_dir, f"best_model_epoch_{epoch}")
            os.makedirs(save_path, exist_ok=True)
            self.model.save_pretrained(save_path)
            self.tokenizer.save_pretrained(save_path)
            torch.save(self.history, os.path.join(save_path, 'training_history.pt'))
            print(f"New best model saved at {save_path}")

            # 최고 모델 경로 업데이트
            self.best_model_path = save_path

    def calculate_metrics(self, predictions, labels):
        predictions = torch.argmax(predictions, dim=-1)
        correct = (predictions == labels).masked_fill(labels == -100, 0)
        accuracy = correct.sum().item() / (labels != -100).sum().item()
        return accuracy

    def train(self, train_dataset, valid_dataset=None):
        start_time = time.time()
        print(f"\nStarting training at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Training parameters:")
        print(f"- Batch size: {self.training_args['batch_size']}")
        print(f"- Learning rate: {self.training_args.get('learning_rate', '2e-5')}")
        print(f"- Max length: {self.max_length}")
        print(f"- Number of epochs: {self.training_args['num_epochs']}")
        print(f"- Training samples: {len(train_dataset)}")
        if valid_dataset:
            print(f"- Validation samples: {len(valid_dataset)}")
        print("\n" + "="*50 + "\n")

        # 데이터셋 전처리
        print("Preprocessing training data...")
        train_dataset = train_dataset.map(
            self.preprocess_data,
            batched=True,
            remove_columns=train_dataset.column_names,
            desc="Processing training data"
        )

        if valid_dataset is not None:
            print("Preprocessing validation data...")
            valid_dataset = valid_dataset.map(
                self.preprocess_data,
                batched=True,
                remove_columns=valid_dataset.column_names,
                desc="Processing validation data"
            )

        # DataCollator 설정
        data_collator = DataCollatorForSeq2Seq(
            tokenizer=self.tokenizer,
            model=self.model,
            padding=True,
            return_tensors="pt"
        )

        # DataLoader 설정 (num_workers=0으로 변경하여 멀티프로세싱 관련 오류 방지)
        train_dataloader = torch.utils.data.DataLoader(
            train_dataset,
            batch_size=self.training_args["batch_size"],
            shuffle=True,
            collate_fn=data_collator,
            num_workers=0,
            pin_memory=True
        )

        if valid_dataset is not None:
            valid_dataloader = torch.utils.data.DataLoader(
                valid_dataset,
                batch_size=self.training_args["batch_size"],
                shuffle=False,
                collate_fn=data_collator,
                num_workers=0,
                pin_memory=True
            )
        best_val_loss = float('inf')
        early_stopping_counter = 0
        early_stopping_patience = self.training_args.get('patience', 3)

        for epoch in range(self.training_args["num_epochs"]):
            epoch_start_time = time.time()
            
            self.model.train()
            epoch_loss = 0
            epoch_accuracy = 0
            train_steps = 0
            
            progress_bar = tqdm(train_dataloader, desc=f"Training Epoch {epoch + 1}")
            batch_losses = []
            batch_accuracies = []
            
            for batch_idx, batch in enumerate(progress_bar):
                input_ids = batch["input_ids"].to(self.device)
                attention_mask = batch["attention_mask"].to(self.device)
                labels = batch["labels"].to(self.device)

                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=labels,
                )
                
                loss = outputs.loss # 손실 계산
                accuracy = self.calculate_metrics(outputs.logits, labels)
                
                loss.backward()# 손실 역전파 
                self.optimizer.step() # 가중치 업데이트
                self.optimizer.zero_grad() # 그래디언트 초기화

                batch_losses.append(loss.item())
                batch_accuracies.append(accuracy)
                
                current_loss = np.mean(batch_losses[-100:])
                current_accuracy = np.mean(batch_accuracies[-100:])
                progress_bar.set_postfix({
                    'loss': f'{current_loss:.4f}',
                    'accuracy': f'{current_accuracy:.4f}',
                    'batch': f'{batch_idx + 1}/{len(train_dataloader)}'
                })

            avg_train_loss = np.mean(batch_losses)
            avg_train_accuracy = np.mean(batch_accuracies)

            if valid_dataset is not None:
                self.model.eval()
                val_losses = []
                val_accuracies = []

                print("\nRunning validation...")
                with torch.no_grad():
                    for batch in tqdm(valid_dataloader, desc="Validating"):
                        input_ids = batch["input_ids"].to(self.device)
                        attention_mask = batch["attention_mask"].to(self.device)
                        labels = batch["labels"].to(self.device)

                        outputs = self.model(
                            input_ids=input_ids,
                            attention_mask=attention_mask,
                            labels=labels,
                        )
                        
                        loss = outputs.loss
                        accuracy = self.calculate_metrics(outputs.logits, labels)
                        
                        val_losses.append(loss.item())
                        val_accuracies.append(accuracy)

                avg_val_loss = np.mean(val_losses)
                avg_val_accuracy = np.mean(val_accuracies)

                if avg_val_loss < best_val_loss:
                    best_val_loss = avg_val_loss
                    early_stopping_counter = 0
                    self.history['best_epoch'] = epoch + 1
                    print(f"\nNew best validation loss: {best_val_loss:.4f}")
                    self.save_model_and_tokenizer(epoch + 1, is_best=True)  # 최고 모델만 저장
                else:
                    early_stopping_counter += 1


            epoch_time = time.time() - epoch_start_time
            self.history['epoch_times'].append(epoch_time)
            self.history['train_loss'].append(avg_train_loss)
            if valid_dataset is not None:
                self.history['val_loss'].append(avg_val_loss)

            # Print epoch summary
            print(f"\nEpoch {epoch + 1} Summary:")
            print(f"Time taken: {epoch_time:.2f} seconds")
            print(f"Average training loss: {avg_train_loss:.4f}")
            print(f"Training accuracy: {avg_train_accuracy:.4f}")
            if valid_dataset is not None:
                print(f"Validation loss: {avg_val_loss:.4f}")
                print(f"Validation accuracy: {avg_val_accuracy:.4f}")
                print(f"Best validation loss so far: {best_val_loss:.4f}")
                print(f"Early stopping counter: {early_stopping_counter}/{early_stopping_patience}")

            if early_stopping_counter >= early_stopping_patience:
                print("\nEarly stopping triggered.")
                break
    def predict(self, text: str) -> str:
        """모델 추론"""
        inputs = self.tokenizer(
            f"키워드 추출: {self._normalize_text(text)}",
            return_tensors="pt",
            max_length=self.max_length,
            truncation=True,
        ).to(self.device)

        outputs = self.model.generate(
            inputs["input_ids"], max_length=self.max_length, num_beams=5
        )
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

    def _normalize_text(self, text: str) -> str:
        return text.strip()

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # 멀티프로세싱 경고 방지

trainer = CustomKeyBERTTrainer(
    model_name="facebook/bart-base", # t5-base     skt/kobart-base-v2
    max_length=128,
    learning_rate=1e-4,
    batch_size=8,
    num_epochs=10,
    gradient_accumulation_steps=8,
    patience=3 # 몇 에폭마다 체크포인트 저장할지
)

if __name__ == "__main__":
    torch.cuda.empty_cache()  # GPU 메모리 초기화
    trainer.train(time_train_dataset, time_val_dataset)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

def test_model(text, saved_model_path, type):

    # 모델과 토크나이저 불러오기
    tokenizer = AutoTokenizer.from_pretrained(saved_model_path, local_files_only=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(saved_model_path, local_files_only=True).to('cpu')
    
    inputs = tokenizer(
        f"키워드 추출: {text}",
        return_tensors="pt",
        max_length=128,
        truncation=True
    ).to("cpu")

    outputs = model.generate(
        inputs["input_ids"],
        max_length=128,
        num_beams=5,
        length_penalty=0.7,
        repetition_penalty=1.2,
        early_stopping=True
    )

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("원래 문장: ", text)
    print(f"{type}키워드 추출: {result}")
    

text = '저희 포스코에서 아침에 쓰레기장 인근 불쾌한 냄새로 인해 불편을 겪고 있어 연락드립니다. 감사합니다. 도시 환경 관리팀입니다. 포스코에서 발생한 산성 냄새 문제와 관련해 상담 드리겠습니다. 최근 포스코에서 1시 산성 냄새이 축축하게 나고 있어 주변 환경 개선 필요로 큰 불편을 겪고 있습니다.\
    현재 포스코에서 발생한 쓰레기장 인근 불쾌한 냄새 문제에 대해 조치 중입니다. 1시에 완료될 예정입니다. 네, 처리가 되도록 부탁드립니다. 네, 알겠습니다. 잘 부탁드립니다. 감사합니다. 빠르게 처리하겠습니다.'
        

print()

test_model(text, '/home/yjtech2/Desktop/yurim/LLM/Pre_processing/smell_keyword/time_models/best_model_epoch_6', 'time')